In [ ]:
%pip install gdown

import gdown
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import scipy.stats as stats
from scipy.stats import norm
from scipy.special import psi
from scipy.optimize import root_scalar

import pandas as pd
import seaborn as sns

import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

# %pip install  kaleido

In [ ]:

file_id = "1lGzAIaqG7CU-RZksujeWYSKGJWCQFPUp"

gdown.download(f"https://drive.google.com/uc?id={file_id}", "Diabetes_db.csv", quiet=False)


Downloading...
From: https://drive.google.com/uc?id=1lGzAIaqG7CU-RZksujeWYSKGJWCQFPUp
To: /content/Diabetes_db.csv
100%|██████████| 23.9k/23.9k [00:00<00:00, 8.82MB/s]


'Diabetes_db.csv'

In [ ]:

df = pd.read_csv('Diabetes_db.csv')


#**Part I**



| Symbol / Variable | Meaning |
|-------------------|---------|
| $$x_i$$           | Individual glucose values |
| $$n$$             | Number of glucose values |
| $$\bar{x}$$       | Sample mean |
| $$s^2$$           | Sample variance |
| $$\psi(a)$$       | Digamma function |

## 1. Method of Moments (MoM)

For a Gamma distribution:

- Mean:  
  $$\mu = ab$$

- Variance:  
  $$\sigma^2 = ab^2$$

Using the sample mean $\bar{x}$ and sample variance $s^2$:

- Shape estimate:  
  $$\hat{a}_{\text{MoM}} = \frac{\bar{x}^2}{s^2}$$

- Scale estimate:  
  $$\hat{b}_{\text{MoM}} = \frac{s^2}{\bar{x}}$$

---

##  2. Maximum Likelihood Estimation (MLE)

The log-likelihood function for the Gamma distribution is:

$$\ell(a, b) = n(a \log b - \log \Gamma(a)) + (a - 1) \sum \log x_i - \frac{1}{b} \sum x_i$$

### MLE Estimation Equations:

- From partial derivative w.r.t. $b$:

  $$\hat{b}_{\text{MLE}} = \frac{\bar{x}}{\hat{a}}$$

- From partial derivative w.r.t. $a$, we get the nonlinear equation:

  $$\log(\hat{a}) - \psi(\hat{a}) = \log(\bar{x}) - \frac{1}{n} \sum \log x_i$$

This equation has **no closed-form solution** and must be solved numerically.

---

In [ ]:
# Data preparation
glucose = df['Glucose'].dropna()
glucose_filtered = glucose[glucose > 0]

n = len(glucose_filtered)
x_bar = np.mean(glucose_filtered)
s2 = np.var(glucose_filtered, ddof=1)

# Method of Moments (MoM)
a_mom = x_bar**2 / s2
b_mom = s2 / x_bar

# MLE
log_x = np.log(glucose_filtered)
log_x_bar = np.log(x_bar)
log_mean = np.mean(log_x)

def mle_equation(a):
    return np.log(a) - psi(a) - (log_x_bar - log_mean)

sol = root_scalar(mle_equation, bracket=[0.1, 100], method='brentq')
a_mle = sol.root
b_mle = x_bar / a_mle

# Gamma PDF values
x = np.linspace(min(glucose_filtered), max(glucose_filtered), 1000)
pdf_mom = stats.gamma.pdf(x, a=a_mom, scale=b_mom)
pdf_mle = stats.gamma.pdf(x, a=a_mle, scale=b_mle)

# === Plot 1: Histogram + MoM only ===
fig1 = go.Figure()
fig1.add_trace(go.Histogram(x=glucose_filtered, nbinsx=30, histnorm='probability density', name='Data', marker_color='skyblue', opacity=0.6))
fig1.add_trace(go.Scatter(x=x, y=pdf_mom, mode='lines', name=f'MoM Gamma({a_mom:.2f}, {b_mom:.2f})', line=dict(color='red')))
fig1.update_layout(title='Gamma Fit using Method of Moments (MoM)', xaxis_title='Glucose', yaxis_title='Density')

# === Plot 2: Histogram + MLE only ===
fig2 = go.Figure()
fig2.add_trace(go.Histogram(x=glucose_filtered, nbinsx=30, histnorm='probability density', name='Data', marker_color='skyblue', opacity=0.6))
fig2.add_trace(go.Scatter(x=x, y=pdf_mle, mode='lines', name=f'MLE Gamma({a_mle:.2f}, {b_mle:.2f})', line=dict(color='green', dash='dash')))
fig2.update_layout(title='Gamma Fit using Maximum Likelihood Estimation (MLE)', xaxis_title='Glucose', yaxis_title='Density')

# === Plot 3: Histogram + Both MoM and MLE ===
fig3 = go.Figure()
fig3.add_trace(go.Histogram(x=glucose_filtered, nbinsx=30, histnorm='probability density', name='Data', marker_color='skyblue', opacity=0.6))
fig3.add_trace(go.Scatter(x=x, y=pdf_mom, mode='lines', name=f'MoM Gamma({a_mom:.2f}, {b_mom:.2f})', line=dict(color='red')))
fig3.add_trace(go.Scatter(x=x, y=pdf_mle, mode='lines', name=f'MLE Gamma({a_mle:.2f}, {b_mle:.2f})', line=dict(color='green', dash='dash')))
fig3.update_layout(title='Gamma Fit Comparison: MoM vs MLE', xaxis_title='Glucose', yaxis_title='Density')

fig1.show()
fig2.show()
fig3.show()



#**Part II**





Let:
- $ n $ = number of observations



Calculate the sample variance with

- $$ s^2 = \frac{1}{n - 1} \sum (x_i - \bar{x})^2 $$



- $$ \text{df} = n - 1 $$ $$ \text{df is the degrees of freedom} $$




Using a significance level $\alpha = 0.05$ (95% confidence level):

- $$ \chi^2_{\alpha/2, df} = \text{Chi-Square critical value at lower tail} $$
- $$ \chi^2_{1 - \alpha/2, df} = \text{Chi-Square critical value at upper tail} $$




The confidence interval for population variance $\sigma^2$ is:

- Lower bound:  
  $$ \frac{(n - 1) s^2}{\chi^2_{1 - \alpha/2}} $$

- Upper bound:  
  $$ \frac{(n - 1) s^2}{\chi^2_{\alpha/2}} $$



In [ ]:
bmi = df["BMI"]
n = len(bmi)

s2 = np.var(bmi, ddof=1)

dfree = n - 1

alpha = 0.05
chi2_lower = stats.chi2.ppf(alpha / 2, dfree)
chi2_upper = stats.chi2.ppf(1 - alpha / 2, dfree)

lower_bound = (dfree * s2) / chi2_upper
upper_bound = (dfree * s2) / chi2_lower

{
    "Sample size (n)": n,
    "Sample variance (s^2)": s2,
    "Degrees of freedom": dfree,
    "Chi2 lower (α/2)": float(chi2_lower),
    "Chi2 upper (1−α/2)": float(chi2_upper),
    "Lower bound": float(lower_bound),
    "Upper bound": float(upper_bound)
}


{'Sample size (n)': 768,
 'Sample variance (s^2)': 62.15998395738257,
 'Degrees of freedom': 767,
 'Chi2 lower (α/2)': 692.1469424045539,
 'Chi2 upper (1−α/2)': 845.640958397576,
 'Lower bound': 56.3793738014489,
 'Upper bound': 68.88234964916714}

#**Part III**



We calculate a 95% confidence interval for the **difference in mean glucose levels** between diabetic and non-diabetic individuals using **Welch’s t-test**, which accounts for unequal variances.


Split into:
- Group 0 (Non-Diabetic): $ X_0 $
- Group 1 (Diabetic): $ X_1 $




Let:
- $ n_0, \bar{x}_0, s_0^2 $ be the size, mean, and variance of group 0  
- $ n_1, \bar{x}_1, s_1^2 $ be the size, mean, and variance of group 1


Since $s_0$ and $s_1$ are not close we had to take a different approach to approximate using Welch's t-test. To get the degrees of freedom(in different variances case) we use the formulas below referred from
### https://en.wikipedia.org/wiki/Welch%27s_t-test

Welch’s formula (for unequal variances):

$$
SE = \sqrt{ \frac{s_0^2}{n_0} + \frac{s_1^2}{n_1} }
$$



$$
df = \frac{ \left( \frac{s_0^2}{n_0} + \frac{s_1^2}{n_1} \right)^2 }{ \frac{s_0^4}{n_0^2(n_0 - 1)} + \frac{s_1^4}{n_1^2(n_1 - 1)} }
$$


Using a 95% confidence level (two-tailed):

$$
t^* = t_{1 - \alpha/2, df}
$$



Let:

$$
\Delta \bar{x} = \bar{x}_1 - \bar{x}_0
$$

Then:

$$
\text{CI} = \Delta \bar{x} \pm t^* \cdot SE
$$



---



This gives a **95% confidence interval** for the difference in average glucose levels between the two groups.



In [ ]:

group_0 = df[df["Outcome"] == 0]["Glucose"]
group_1 = df[df["Outcome"] == 1]["Glucose"]


n0, n1 = len(group_0), len(group_1)
mean0, mean1 = group_0.mean(), group_1.mean()
var0, var1 = group_0.var(ddof=1), group_1.var(ddof=1)


se_diff = np.sqrt((var0 / n0) + (var1 / n1))


df_welch = (var0 / n0 + var1 / n1) ** 2 / ((var0**2 / (n0**2 * (n0 - 1))) + (var1**2 / (n1**2 * (n1 - 1))))


t_crit = stats.t.ppf(1 - 0.025, df_welch)


diff_means = mean1 - mean0
lower_bound = diff_means - t_crit * se_diff
upper_bound = diff_means + t_crit * se_diff

{
    "Mean (Diabetic)": float(mean1),
    "Mean (Non-Diabetic)": float(mean0),
    "Difference in Means": float(diff_means),
    "Standard Error": float(se_diff),
    "Degrees of Freedom": float(df_welch),
    "t Critical Value": float(t_crit),
    "95% CI Lower Bound": float(lower_bound),
    "95% CI Upper Bound": float(upper_bound)
}


683.3623246492986
1020.1394572083399


{'Mean (Diabetic)': 141.25746268656715,
 'Mean (Non-Diabetic)': 109.98,
 'Difference in Means': 31.277462686567148,
 'Standard Error': 2.2744703034486986,
 'Degrees of Freedom': 461.3316563534082,
 't Critical Value': 1.9651194976678963,
 '95% CI Lower Bound': 26.807856746393494,
 '95% CI Upper Bound': 35.7470686267408}

#**Part IV**

# One-Sample Proportion Test for $H_0: p \leq 0.5$ vs $H_1: p > 0.5$

## Data Structure and Notation
- We observe binary (Bernoulli) outcomes, where each observation is either a success (1) or failure (0).
- Let $p$ denote the true probability of success in the population.
- Let $X_1, X_2, \dots, X_n$ represent $n$ independent binary observations.
- Let $X = \sum_{i=1}^{n} X_i$ represent the total number of successes.

## Hypothesis Framework
- **Null hypothesis**: $H_0: p \leq 0.5$
- **Alternative hypothesis**: $H_1: p > 0.5$
- **Significance level**: $\alpha = 0.05$

## Test Statistics and Procedures

### 1. Z-test for Proportion
The Z-test uses the normal approximation to the binomial distribution:

1. Calculate the sample proportion: $\hat{p} = \frac{X}{n}$

2. Under the null hypothesis ($p = p_0 = 0.5$), the standard error is:  
   $$
   SE = \sqrt{\frac{p_0 (1 - p_0)}{n}}
   $$

3. Compute the test statistic:  
   $$
   Z = \frac{\hat{p} - p_0}{SE}
   $$

4. Calculate the p-value for an upper-tailed test:  
   $$
   \text{p-value} = 1 - \Phi(Z)
   $$
   where $\Phi$ is the cumulative distribution function of the standard normal distribution.

5. **Decision rule**: Reject $H_0$ if p-value < $\alpha$

### 2. Exact Binomial Test
The exact test uses the binomial probability mass function directly:

1. Calculate the observed number of successes $X$

2. Calculate the p-value as the probability of observing $X$ or more successes under the null hypothesis:  
   $$
   \text{p-value} = P(X \geq x \mid n, p_0) = \sum_{k = x}^{n} \binom{n}{k} p_0^k (1 - p_0)^{n - k}
   $$

3. **Decision rule**: Reject $H_0$ if p-value < $\alpha$

## Confidence Interval

A $(1 - \alpha) \times 100\%$ confidence interval for $p$ provides a range of plausible values:

1. Using the normal approximation:  
   $$
   \hat{p} \pm z_{1 - \alpha/2} \cdot \sqrt{\frac{\hat{p} (1 - \hat{p})}{n}}
   $$

   where $z_{1 - \alpha/2}$ is the $(1 - \alpha/2)$ quantile of the standard normal distribution.

2. **Interpretation**: If the confidence interval lies entirely above 0.5, this provides additional evidence against $H_0$.

## Assumptions

1. The observations are independent and identically distributed.
2. For the Z-test, the normal approximation is valid when $np_0 \geq 10$ and $n(1 - p_0) \geq 10$.

## Additional Considerations

- The Z-test is most appropriate for large samples when the normal approximation is valid.
- The exact binomial test is preferred when sample sizes are small or the normal approximation is questionable.
- The confidence interval provides an additional perspective by showing the precision of our estimate.


In [ ]:



np.random.seed(42)

# Generate binary data with p = 0.55
df = pd.DataFrame({'Outcome': np.random.binomial(1, 0.55, size=300)})

# Summary statistics
successes = df["Outcome"].sum()  # Count actual successes
p_hat = successes / len(df)
n = len(df)
p0 = 0.5  # Null hypothesis value
alpha = 0.05

# Z-test for proportion
se = np.sqrt(p0 * (1 - p0) / n)  # SE under the null
z_stat = (p_hat - p0) / se
p_value_z = 1 - stats.norm.cdf(z_stat)

# Exact binomial test
binom_test = stats.binomtest(successes, n, p=p0, alternative='greater')

# Confidence Interval (using sample proportion for CI)
z_critical = stats.norm.ppf(1 - alpha)
margin_error = z_critical * np.sqrt(p_hat * (1 - p_hat) / n)
ci_lower = max(0, p_hat - margin_error)
ci_upper = min(1, p_hat + margin_error)

# Check if normal approximation is valid
normal_approx_valid = (n * p0 >= 10) and (n * (1 - p0) >= 10)

# Display results
print(f"Sample proportion (p̂): {p_hat:.4f}")
print(f"Standard Error:{se}")
print(f"Number of successes: {successes} out of {n}")
print(f"Null hypothesis (p₀): {p0}")
print(f"Z statistic: {z_stat:.4f}")
print(f"P-value (Z-test): {p_value_z:.4f}")
print(f"P-value (Exact Binomial Test): {binom_test.pvalue:.4f}")
print(f"95% CI for true proportion: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"Normal approximation valid? {normal_approx_valid}")


print("\nConclusion:")
if p_value_z < alpha:
    print(f"Reject H₀ at α = {alpha}: There is significant evidence that p > 0.5 (Z-test p-value = {p_value_z:.4f}).")
else:
    print(f"Fail to reject H₀ at α = {alpha}: No significant evidence that p > 0.5 (Z-test p-value = {p_value_z:.4f}).")

if binom_test.pvalue < alpha:
    print(f"The exact binomial test also leads to rejecting H₀ (p-value = {binom_test.pvalue:.4f}).")
else:
    print(f"The exact binomial test also fails to reject H₀ (p-value = {binom_test.pvalue:.4f}).")

print(f"\nThe estimated proportion is {p_hat:.4f} with 95% CI [{ci_lower:.4f}, {ci_upper:.4f}].")
if ci_lower > p0:
    print(f"Since the confidence interval is entirely above {p0}, this provides further evidence against H₀.")
else:
    print(f"Since the confidence interval contains {p0}, this is consistent with the possibility that p could be ≤ 0.5.")

Sample proportion (p̂): 0.5500
Standard Error:0.02886751345948129
Number of successes: 165 out of 300
Null hypothesis (p₀): 0.5
Z statistic: 1.7321
P-value (Z-test): 0.0416
P-value (Exact Binomial Test): 0.0470
95% CI for true proportion: [0.5028, 0.5972]
Normal approximation valid? True

Conclusion:
Reject H₀ at α = 0.05: There is significant evidence that p > 0.5 (Z-test p-value = 0.0416).
The exact binomial test also leads to rejecting H₀ (p-value = 0.0470).

The estimated proportion is 0.5500 with 95% CI [0.5028, 0.5972].
Since the confidence interval is entirely above 0.5, this provides further evidence against H₀.
